A lot of the code across these notebooks is duplicated, however handling each bit of data varies. As a result, each is independent, although we could've made one function to read in files and one to parse them. 

In [1]:
import pandas as pd
import numpy as np
import csv 
import os 
import re

In [2]:
CODEFILE = './nhgis0015_ds92_1960_tract_codebook.txt'
DATAFILE = './nhgis0015_ds92_1960_tract.csv'
OUTPUTFILE = './1960s_fordb.csv'

pd.set_option("display.max_rows",101)
pd.set_option("display.max_columns",101)
pd.set_option("display.max_seq_items",500)

In [3]:
f = open(CODEFILE, 'r')

# keep all lines that are not whitespace
lines = [line for line in f.readlines() if not line is ""]

In [4]:
# regex to save lines that start with #.,  "NHGIS code:"
tab = []
code = []
mini_code = {}

# create column names from regex dictionary
for line in lines:
    if re.match('[0-9]+', line):
        temp = re.sub('[0-9]*\. ', '', line)
        temp = re.sub(' ', '_', temp)
        tab.append(temp[:-1])
    if re.match('NHGIS code:', line):
        code.append(line[-4:-1])
    if re.match('.*[0-9]{3}:', line):
        preproc = re.sub(' ', '', line)
        preproc = re.sub('\n', '', preproc)
        c, n = preproc.split(":")[0], "_".join(preproc.split(":")[1:])
        threeletter = c[:3]
        store = mini_code.get(threeletter, {})
        store[c] = n
        mini_code[threeletter] = store

In [5]:
# construct a dictionary of code to table
code_dict = {}

for i, c in enumerate(code):
    code_dict[c] = tab[i]

In [6]:
# convert the pd cols from NHGIS names
data = pd.read_csv(DATAFILE)

# only keep IL
working_df = data[(data.STATE == 'Illinois') & (data.COUNTY == 'Cook')]

In [7]:
# get new names 
new_cols = []
for colname in working_df.columns:
    if colname[:3] in code_dict.keys():
        sub_cat_dict = mini_code.get(colname[:3], {}) 
        subcat = sub_cat_dict[colname]
        keep = code_dict[colname[:3]]+"_"+ subcat
    else: 
        keep = colname
    new_cols.append(keep)

In [8]:
# rename columns
working_df.columns = new_cols

In [11]:
# calculate and rename columns

final_df = working_df[['GISJOIN', 'YEAR', 'TRACTA', 'COUNTY', 'STATE']]

# race
final_df = final_df.assign(Totalpop = working_df['Total_Persons_Total'])
final_df = final_df.assign(Pwhite = (final_df.Totalpop - working_df.Total_Nonwhite_Population_Total)/final_df.Totalpop)
final_df = final_df.assign(Pblack = (working_df['Nonwhite_Persons_by_Race_by_Sex_Negroes>>Male'] +
                                     working_df['Nonwhite_Persons_by_Race_by_Sex_Negroes>>Female'])/final_df.Totalpop)
final_df = final_df.assign(Pnonwhite = (working_df['Nonwhite_Persons_by_Race_by_Sex_OtherNonwhites>>Male']+
                                     working_df['Nonwhite_Persons_by_Race_by_Sex_OtherNonwhites>>Female'])/final_df.Totalpop)

# units
final_df = final_df.assign(Totalunits = working_df['Total_House_Units_[from_printed_report]_Total'])

Vac = (working_df['Housing_Units_by_Vacancy_[from_printed_report]_Availablevacant'] + 
                          working_df['Housing_Units_by_Vacancy_[from_printed_report]_Othervacant']) / final_df.Totalunits

final_df = final_df.assign(Poccupied = 1 - Vac)
final_df = final_df.assign(Pvacant = Vac)

Allowner = working_df['Occupied_Housing_Units_by_Tenure_[from_printed_report]_Owner-occupied']
final_df = final_df.assign(Powner = Allowner / (final_df.Poccupied*final_df.Totalunits)) #check not just of non-vacant

Allrenter = working_df['Occupied_Housing_Units_by_Tenure_[from_printed_report]_Renter-occupied']
final_df = final_df.assign(Prented = Allrenter / (final_df.Poccupied*final_df.Totalunits))

final_df.fillna("NaN")

,GISJOIN,YEAR,TRACTA,COUNTY,STATE,Totalpop,Pwhite,Pblack,Pnonwhite,Totalunits,Poccupied,Pvacant,Powner,Prented
5110,G17003100001,1960,1,Cook,Illinois,4237.0,0.989379,0.000236016,0.0059004,1698.0,0.957008,0.0429918,0.158154,0.841846
5111,G17003100002,1960,2,Cook,Illinois,1510.0,1,0,0.00331126,477.0,0.981132,0.0188679,0.239316,0.760684
5112,G17003100003,1960,3,Cook,Illinois,6622.0,0.996074,0.000604047,0.00453035,2793.0,0.964554,0.0354458,0.143281,0.856719
5113,G17003100004,1960,4,Cook,Illinois,8051.0,0.990809,0.000869457,0.00857036,3348.0,0.93399,0.0660096,0.159578,0.840422
5114,G17003100005A,1960,5,Cook,Illinois,9596.0,0.989162,0.0010421,0.0100042,4410.0,0.931293,0.0687075,0.0898466,0.910153
5115,G17003100005B,1960,5,Cook,Illinois,5074.0,0.996847,0.00177375,0.000788333,2087.0,0.93196,0.0680402,0.105398,0.894602
5116,G17003100006,1960,6,Cook,Illinois,8614.0,0.996633,0.00058045,0.00510796,3496.0,0.962243,0.0377574,0.156956,0.843044
5117,G17003100007,1960,7,Cook,Illinois,5821.0,0.99519,0.00292046,0.00360763,2732.0,0.943265,0.056735,0.0997284,0.900272
5118,G17003100008,1960,8,Cook,Illinois,7363.0,0.991851,0.000543257,0.00461768,3482.0,0.938254,0.0617461,0.0514233,0.948577
5119,G17003100009,1960,9,Cook,Illinois,11753.0,0.998554,0.000935931,0.0022122,4071.0,0.989192,0.0108082,0.409983,0.590017


In [12]:
# rename and export
column_names = ['GISJOIN','YEAR', 'TRACTA', 'COUNTY','STATE', 'Total Pop', '% White', 
               '% Black', '% Other races', 'Total Units', '% Occupied', 
               '% Vacant', '% Owner Occupied', '% Renter']

final_df.to_csv(path_or_buf=OUTPUTFILE, index=False, header=column_names)

In [13]:
# check the fill of None
final_df[final_df.Poccupied == None]

,GISJOIN,YEAR,TRACTA,COUNTY,STATE,Totalpop,Pwhite,Pblack,Pnonwhite,Totalunits,Poccupied,Pvacant,Powner,Prented
